In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

import torch as t
import einops
from typing import Dict, List
import torch.sparse as sparse
from tqdm import tqdm
import gc
import torch as t
import torch.sparse as sparse
import einops
from tqdm import tqdm
import pickle
from typing import Dict, List
from pathlib import Path

from trainers.scae import SCAESuite
from buffer import AllActivationBuffer
from utils import load_model_with_folded_ln2, load_iterable_dataset
from find_top_connections import get_importance_scores
from trainers.top_k import AutoEncoderTopK
from trainers.scae import SCAESuite, TrainerSCAESuite, TrainerConfig

DTYPE = t.bfloat16
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
t.manual_seed(42)
t.set_grad_enabled(True)
gc.collect()

0

In [2]:
model = load_model_with_folded_ln2("gpt2", device=device, torch_dtype=DTYPE)
data = load_iterable_dataset('Skylion007/openwebtext')
suite = SCAESuite.from_pretrained(
    'jacobcd52/gpt2_suite_folded_ln',
    device=device,
    dtype=DTYPE,
    )

initial_submodule = model.transformer.h[0]
layernorm_submodules = {}
submodules = {}
for layer in range(model.config.n_layer):
    submodules[f"mlp_{layer}"] = (model.transformer.h[layer].mlp, "in_and_out")
    submodules[f"attn_{layer}"] = (model.transformer.h[layer].attn, "out")

    layernorm_submodules[f"mlp_{layer}"] = model.transformer.h[layer].ln_2

buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodules,
    initial_submodule=initial_submodule,
    layernorm_submodules=layernorm_submodules,
    d_submodule=model.config.n_embd,
    n_ctxs=128,
    out_batch_size = 8,
    refresh_batch_size = 256,
    device=device,
    dtype=DTYPE,
)

/root/dictionary_learning/notebooks/../trainers/scae.py:765: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = t.load(checkpoint_path, map_location='cpu')
/root/dic

In [3]:
with open("/root/dictionary_learning/connections_100.pkl", "rb") as f:
    suite.connections = pickle.load(f)

In [4]:
trainer_cfg = TrainerConfig(
    steps=1000,
    connection_sparsity_coeff=0.0,
    lr_decay_start_proportion=0.8,
    dead_feature_threshold=10_000_000,
    base_lr=2e-4,
    n_threshold=10,
    n_random=10,
)

In [5]:
trainer = TrainerSCAESuite(
    suite,
    config=trainer_cfg,
)

In [6]:
initial_acts, input_acts, target_acts, ln_scales = next(buffer)

In [7]:
trainer.update(
    0,
    initial_acts,
    input_acts,
    target_acts,
    ln_scales)

124416.0

In [10]:
pruned_out = suite.pruned_forward_train(
    initial_acts, #.unsqueeze(1).repeat(1, 2, 1),
    {k: v for k, v in input_acts.items()},
    {k: v for k, v in ln_scales.items()},
    n_random=10,
    n_threshold=11,
)

In [12]:
suite.get_ce_loss(initial_acts, pruned_out)